In [1]:
import requests
import json
import csv
from datetime import datetime

In [16]:
def api_tester(dish_name):
    now = datetime.now()
    dt_string = now.strftime("%d_%m_%Y_%H_%M_%S")

    api_base = "https://api.edamam.com/search?"

  # Level up:
  # Implement lemmatization using trained dataset on input in order to make 
  # future database be less likely to have redundant entries 
  # (e.g., taco vs tacos)

    q = f"q={dish_name}"

  # Level up:
  # Check a database of dishes to see if this query has been asked for already
  # If not, do an API call

  # Currently, just does an API call, may hit API limit if continuing with this
  # version
    f = open("../secrets/edamam.json","r")
    cred = json.load(f)
    f.close()

    app_id = cred["id"]
    app_id_s = f"&app_id={app_id}"

    app_key = cred["key"]
    app_key_s = f"&app_key={app_key}"
    
    api_call = api_base + q+ app_id_s + app_key_s #+ limiter

    resp = requests.get(api_call)

    if resp.status_code == 200:
        response_dict = resp.json()
        resp_dict_hits = response_dict['hits']

        # Store the API result into a JSON and the cuisine type and dish name into a 
        # csv
        # Heroku does not save files to directory
        # Can work with EC2
        with open(f"../write_data/{dt_string}_{dish_name}_edamam_api_return.json", "w") as f:
            json.dump(resp_dict_hits, f)
            
        urls = []
        labels = []
        sources = []
        ingreds = []

        for recipe in resp_dict_hits:
            recipe_path = recipe['recipe']
            urls.append(recipe_path['url'])
            labels.append(recipe_path['label'])
            sources.append(recipe_path['source'])
            ingreds.append([item['text'] for item in recipe_path['ingredients']])

        all_recipes = {'url': urls,
                      'label': labels, 
                      'source': sources, 
                      'ingredients': ingreds
                      }
        
        return all_recipes
    else:
        return("Error, unable to retrieve. Server response code is: ", 
              resp.status_code)


In [17]:
grap_sorb = api_tester("grapefruit sorbet")

In [18]:
grap_sorb

{'url': ['http://www.marthastewart.com/318276/grapefruit-sorbet',
  'http://www.jamieoliver.com/recipes/fruit-recipes/campari-grapefruit-sorbet',
  'https://food52.com/recipes/3038-pink-grapefruit-sorbet',
  'http://www.myrecipes.com/recipe/pink-grapefruit-sorbet',
  'http://www.delish.com/cooking/recipe-ideas/recipes/a25069/pink-grapefruit-sorbet-1313/',
  'http://www.williams-sonoma.com/recipe/ruby-grapefruit-sorbet.html',
  'http://gluttonforlife.com/2010/02/23/ice-ice-baby/',
  'https://www.foodnetwork.com/recipes/pink-grapefruit-sorbet-recipe0-1914080',
  'http://www.marthastewart.com/318267/red-grapefruit-sorbet',
  'http://www.davidlebovitz.com/2011/04/grapefruit-campari-sorbet/'],
 'label': ['Grapefruit Sorbet',
  'Campari & Grapefruit Sorbet',
  'Pink Grapefruit Sorbet',
  'Pink Grapefruit Sorbet',
  'Pink Grapefruit Sorbet',
  'Ruby Grapefruit Sorbet',
  'Pink Grapefruit Sorbet',
  'Pink Grapefruit Sorbet',
  'Red Grapefruit Sorbet',
  'Grapefruit-Campari Sorbet'],
 'source':